In [ ]:
import zap
import pypsa
from zap.tests import network_examples as examples
from zap.importers import pypsa

In [ ]:
pn = examples.load_example_network('texas_7node')

In [ ]:
net, devices = zap.importers.load_pypsa_network(pn, power_unit=1e3, cost_unit=10.0)

In [ ]:
net.dispatch(devices=devices, time_horizon=48)